## 웹 크롤링

### 인터넷 접속 라이브러리 추가 

- urllib.request 모듈로 작업 
- html 소스 로드로 별다른 문제 없음 

In [1]:
from urllib.request import Request, urlopen

# 도시별 날씨 검색 함수 
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url= 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page= urlopen(url=url)
    
    text= page.read().decode('utf-8')      # 페이지 읽기 위해서 
    text=text[text.find(f'>{city}</a>'):]  # city 부분을 </a> 전 까지 찾아라 (?)

    # 기온 가져오기 → 반복문 13번 
    for i in range(7):                     # 7번 반복   
        text= text[text.find('<td>')+1:]   # td에서 +1 한 인덱스를 text로 저장(?)
    
    strat= 3                      # td는 하나의 인덱스
    end= text.find('</td>')       # td 닫기 전까지 찾아라 
    current_temp=text[strat:end]  # 3번째 인덱스부터 td 닫기 전까지 현재 기온으로 저장 
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 습도  가져오기
    for i in range(3):
        text= text[text.find('<td>')+1:]
    
    strat= 3
    end= text.find('</td>')
    current_humid=text[strat:end]
    print(f'{city}의 현재 기온은 {current_humid}% 입니다.')

get_weather('부산')

부산의 현재 기온은 11.4˚C 입니다.
부산의 현재 기온은 23% 입니다.


*내가 인덱스 볼려고 따로 펼쳐놓음* 
```python
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url= 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page= urlopen(url=url)
    
    text= page.read().decode('utf-8')
    print(text[text.find(f'>{city}</a>'):])
get_weather('부산')
```
```python
# def getRequestUrl(url):
#     req= Request(url)

#     try :
#         res = urlopen(req)
#         if res.getcode() == 200:
#             return res.read().decode('utf-8')
#     except  Exception as e:
#         print(e)
#         return None

 # api_url+='?serviceKey=4n4Miwzm5p37SLb9Jk9bJa%2FMhFYSTl8mkQIensYxsOuwWyjpePzkk6oyRp3pOsd8GVnzwwQelKHMwSc0bPVfSA%3D%3D'
    # api_url+='&pageNo=1'
    # api_url+='&numOfRows=10'
    # api_url+=f'&resultType={type}'
    # api_url+=f'&stationLoc={quote(stationName)}'

    # response = getRequestUrl(api_url)

    # if response == None :
    #     return None
    # else:
    #     return json.loads(response)


# quote('롯데백화점')    
```

### OpenAPI 크롤링

- 공공데이터 포털 :
    - https://www.data.go.kr/
    - 부산광역시 버스정보안내기 현황

```python
    response = requests.get(total_url, verify=False)
```
- 문제점 
    - https 를 일반 request로 부르면 SSL 오류 발생
    - 해결하려면 외부 모듈 requests를 사용, verify=False 옵션을 지정
    - 아래 셀의 19번 라인!

In [8]:
import requests
from urllib.parse import quote, unquote, urlencode   # 한글을 urlencode로 변환 해줌  롯데 ->'%EB%A1%AF%EB%8D%B0%EB%B0%B1%ED%99%94%EC%A0%90' 
import json


#stationName 정류소 이름 
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" +urlencode( 
        {
            'serviceKey': '4n4Miwzm5p37SLb9Jk9bJa/MhFYSTl8mkQIensYxsOuwWyjpePzkk6oyRp3pOsd8GVnzwwQelKHMwSc0bPVfSA==',
            'pageNO': '1',
            'numOfRows':'10',
            'resultType':type,
            'stationLoc': stationName
        }
    )
    total_url = api_url +queryString
    response = requests.get(total_url, verify=False)
    return response.text

   
try:
    result = getDataPortalSearch('백화점', 'json')
    json_data = json.loads(result)
    
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    print(station_data)

    for i in station_data:
        print(i)
except Exception as e:
    print('찾는 데이터가 없습니다.')



[{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}, {'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}, {'stationNum': '05718', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15776525', 'lng': '129.0566948', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}, {'stationNum': '05719', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15748116', 'lng': '129.0561774', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}, {'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}, {'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}]
{'stationNum': '09283'

c:\Source\studyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시 

```python
stop_str='<h4>'+item['stationLoc']+'</h4>'+item['stationNum']+'<br>'+item['addr']
```
- 문제점 
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 안됨 !
    - 구식 방법인 문자열 결합방식으로 해결 

In [10]:
import folium

if len (station_data) > 0:
    center_lat=station_data[0]['lat']
    center_lng=station_data[0]['lng']

m = folium.Map(location=[center_lat,center_lng], zoom_start=13)

for item in station_data:
    stop_str='<h4>'+item['stationLoc']+'</h4>'+item['stationNum']+'<br>'+item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe , min_width=200, max_width=200)         # 팝업 사이즈 지정 
    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                  icon=folium.Icon(icon='pushpin')).add_to(m)

m

### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구 (라이브러리)

```python
pip install beautifulsoup4
```

In [4]:
!pip install beautifulsoup4


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

```python
from bs4 import BeautifulSoup

In [7]:
from bs4 import BeautifulSoup
import requests


url= 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error  : {response.status_code}')

제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
